<a href="https://colab.research.google.com/github/Bhandari007/sequence_model_course/blob/main/w2a2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading starter files

In [9]:
# !wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week2/w2a2/emo_utils.py
# !wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week2/w2a2/generateTestCases.py
# !wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week2/w2a2/test_utils.py
# !wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week2/w2a2/data/emojify_data.csv -P /data/
# !wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week2/w2a2/data/fake_glove.6B.50d.txt -P /data/

# !wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week2/w2a2/data/test_emoji.csv -P /data/
# !wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week2/w2a2/data/train_emoji.csv -P /data/
!wget https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week2/w2a2/data/tesss.csv -P /data/

--2022-10-20 11:06:42--  https://raw.githubusercontent.com/abdur75648/Deep-Learning-Specialization-Coursera/main/Sequence%20Models/week2/w2a2/data/tesss.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1481 (1.4K) [text/plain]
Saving to: ‘/data/tesss.csv’

tesss.csv           100%[===================>]   1.45K  --.-KB/s    in 0s      

2022-10-20 11:06:42 (15.8 MB/s) - ‘/data/tesss.csv’ saved [1481/1481]



In [5]:
import os
os.listdir("/data/")

['train_emoji.csv',
 'emojify_data.csv',
 'test_emoji.csv',
 'fake_glove.6B.50d.txt']

# Emojify!

In this notebook, we're going to use word vector representations to build an Emojifier.

We'll implement a model which inputs a sequence and finds the most appropriate emoji to be used with that sentence.

**Objectives:**

* Create an embedding layer in Keras with pre-trained word vectors.
* Explain the advantages and disadvantages of the GloVe algorithm.
* Describe how negative sampling learns word vectors more efficiently than other methods.
* Build a sentiment classifier using word embedding
* Build and train a more sophisticated classifier using LSTM

# Packages

In [7]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 8.2 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=5e760902b736c07af388667360211258aa3af8fede30a250114950ff61076a49
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


In [21]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt
from test_utils import *

%matplotlib inline

# 1 - Baseline Model: Emojifier-V1

# 1.1 - Dataset EMOJISET

We will start by buildinga baseline classifier

We have a tiny dataset (X,Y) where:
  * X contains 127 sentences (strings).
  * Y contains an integer label between 0 and 4 corresponding to an emoji for each sentence

Load the dataset. The dataset is split between training (127 examples) and testing (56 examples).


In [15]:
X_train, Y_train = read_csv("/data/train_emoji.csv")
X_test, Y_test = read_csv("/data/tesss.csv")

In [17]:
maxlen = len(max(X_train, key = len).split())
maxlen

10

## 1.2 - Overview of the Emojifier- V1

In this section, we'll implement a baseline model called "Emojifier-v1".

### Inputs and Outputs
* The input of the model is a string corresponding to a sentence (e.g "i love you").
* The output will be a probably vector of shape (1,5) (indicating that there are 5 emojis to choose from).
* The (1,5) probability vector is passed to an argmax layer, which extracts the index of the emoji with the highest probability.

### One-hot encoding
* To get lables into a format suitable for training a softmax classifier, convert Y from its current shape (m,1) into a "one-hot representation" (m,5).
  * Each row is a one-hot vector giving the label of one example.
  * Here, `Y_oh` stands for "Y-one-hot" in the variable names `Y_oh_train` and `Y_oh_test`:

In [23]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)